In [1]:
import sys
import time
from flask_restful import fields, marshal, reqparse, Resource, inputs

import pandas as pd
import geopandas as gpd
import requests

BASE_URL = 'https://api.mountainhub.com/timeline'
HEADER = { 'Accept-version': '1' }
ONE_MONTH = 2592000000


import datetime
now_ts = datetime.datetime.now().timestamp()*1000
start_ts = datetime.datetime(2019,12,29).timestamp()*1000

In [2]:
def parse_data(record):

    cso_format = {
        'author_name' : fields.String(attribute='actor.full_name') or fields.String(attribute='actor.fullName'),
        'id' : fields.String(attribute=lambda x: x['_id']),
        'timestamp' : fields.Integer(attribute=lambda x: x['reported_at']),
        'lat' : fields.Float(attribute=lambda x: x['location']['coordinates'][1]),
        'long' : fields.Float(attribute=lambda x: x['location']['coordinates'][0]),
        'snow_depth' : fields.Float(attribute=lambda x: x['observation']['details'][0]['snowpack_depth']),
        'source' : fields.String(default = 'MountainHub')
    }

    return marshal(record, cso_format)

In [3]:
def fetch_raw_data(min_timestamp, max_timestamp):

    args = {
      'publisher': 'all',
      'limit': 10000,
      'since' : min_timestamp,
      'before' : max_timestamp,
    }

    response = requests.get(BASE_URL, params=args, headers=HEADER)
    data = response.json()

    #if 'results' not in data or len(data['results']) == 0:
      #  return empty_cso_dataframe()
    #else:
    results = data['results']
    #all_results = [ parse_data(result) for result in results ]
    #valid_results = [result for result in all_results if result is not None]
#    valid_results = with_elevation(valid_results)
#        if len(valid_results) == 0:
#            return empty_cso_dataframe()

#        df = pd.DataFrame.from_records(valid_results)
    return results

In [4]:
df = fetch_raw_data(start_ts, now_ts )

In [7]:
cso_format = {
        'author_name' : fields.String(attribute='actor.full_name') or fields.String(attribute='actor.fullName'),
        'id' : fields.String(attribute=lambda x: x['_id']),
        #'timestamp' : fields.Integer(attribute=lambda x: x['observation']['reported_at']),
        'lat' : fields.Float(attribute=lambda x: x['location']['coordinates'][1]),
        'long' : fields.Float(attribute=lambda x: x['location']['coordinates'][0]),
        #'snow_depth' : fields.Float(attribute=lambda x: x['observation']['details'][0]['snowpack_depth']),
        'source' : fields.String(default = 'MountainHub')}

In [8]:
test = marshal(df, cso_format)
print(test)

KeyError: 'observation'

In [29]:
df[0]['observation']['details'][0]['snowpack_depth']

'35'

In [9]:
df[0]['observation']['reported_at']

1577733250329